# Кейс А11, 1 часть

## А11.2.2 Обработка данных

Импортируем нужные библиотеки

In [1]:
import pandas as pd

Определяю константы, которые мне понадобятся.

In [2]:
TESTS_FILE='Data1/ab_test_groups.csv.bz2'
PAYMENTS_FILE='Data1/payments.csv.bz2'
TEST_ID=127
START_DATE='2019-08-05'
END_DATE='2019-08-12'  # This day will not be included

Первый файл — разбивка пользователей по тестам и группам, из него нам интересен только тест 127.  Сразу проверю, что группы пользователей A и B не пересекаются.

In [3]:
test_uids_df = pd.read_csv(TESTS_FILE)
test_uids_df = test_uids_df[test_uids_df.ab_test_id==127].reset_index()
group_a_st = set(test_uids_df.loc[test_uids_df.grp == 'A', 'user_id'])
group_b_st = set(test_uids_df.loc[test_uids_df.grp == 'B', 'user_id'])
print(group_a_st.intersection(group_b_st))

set()


Множества не пересекаются, отлично. Смотрим, как распределены пользователи по группам

In [4]:
print("В группе A {} пользователей, в группе B {}.".format(len(group_a_st), len(group_b_st)))

В группе A 76605 пользователей, в группе B 76627.


Соотношение около 50/50, количество пользователей в группах достаточно приличное, чтобы считать статистику. 

## Считаем статистику по контрольной и экспериментальной группам

Загружаем таблицу платежей, вырезаем интересующий нас временной диапазон.

In [5]:
payments_df = pd.read_csv(PAYMENTS_FILE)
payments_df = payments_df.loc[payments_df.created_at.between(START_DATE, END_DATE, inclusive=False)]

Готовим набор данных для обработки тестирования, вырезаем только колонки `user_id` и `price`.  Проверяем, что пользователи встречаются в таблице не более одного раза (то есть совершают в исследуемый интервал только одну покупку)

In [6]:
abtest_df = payments_df.loc[:,('user_id', 'price')]
assert(len(abtest_df) == abtest_df.user_id.nunique())

Соединяем таблицы групп пользователей и покупок, избавляемся от лишних полей, считаем интересные нам показатели:

In [7]:
abtest_df = payments_df.loc[:,('user_id', 'price')]
abtest_df = test_uids_df.merge(abtest_df, on='user_id', how='left')
abtest_df = abtest_df.loc[:,['user_id','grp','price']]
abtest_df = abtest_df.set_index('user_id')
print("Количество покупателей в контрольной (А) и экспериментальной (B) группах:")
purch_cnt = (abtest_df.groupby('grp', as_index = False)['price'].count()
            ).rename({'price':'buyers'}, axis=1)
display(purch_cnt)

Количество покупателей в контрольной (А) и экспериментальной (B) группах:


,grp,buyers
0,A,4279
1,B,9427


Разница больше чем вдвое. Для расчёта конверсии и показа разбиения на группы нам понадобится исходное количество
пользователей в каждой группе.  Расчитаем доли пользователей по группам, доли покупателей по группам и конверсии.

In [8]:
users_and_buyers = abtest_df.reset_index().groupby('grp').count().rename(
    {'user_id': 'users_count', 'price': 'buyers'},axis=1)
users_and_buyers['users_share'] = users_and_buyers['users_count']/sum(users_and_buyers['users_count'])
users_and_buyers['buyers_share'] = users_and_buyers['buyers'] / sum(users_and_buyers['buyers'])
users_and_buyers['conversion'] = users_and_buyers['buyers'] / users_and_buyers['users_count']
display(users_and_buyers)

,users_count,buyers,users_share,buyers_share,conversion
grp,,,,,
A,76605,4279,0.499928,0.312199,0.055858
B,76627,9427,0.500072,0.687801,0.123025


Разделение на группы (A/B) показано в колонке users_share, оно до 3 знака после запятой соответствует разделению 50/50.  При этом конверсия в группе A около 6%, а в группе B более 12%, то есть она выросла больше, чем вдвое.
Аналогично видно, что более 2/3 покупателей пришли из группы B.

Посмотрим, что при этом происходит со средним чеком.

In [9]:
buyers_only_abtest = abtest_df[abtest_df.price.notnull()]
buyers_only_abtest.head()

,grp,price
user_id,,
1,B,140.0
10,B,140.0
23,B,140.0
24,B,700.0
39,B,140.0


В датафрейме `buyers_only_abtest` содержатся только покупатели, колонки: `user_id`, `grp` и цена. Для расчёта среднего чека по группам достаточно сгруппировать датафрейм по полю `grp` и осреднить цену.

In [10]:
avc = (buyers_only_abtest.groupby('grp', as_index=False)['price'].mean().rename({'price': 'avg_cheque'}, axis=1 ))
display(avc)
print("Снижение среднего чека: {:.2%}".format(1-(avc.avg_cheque[1] / avc.avg_cheque[0])))

,grp,avg_cheque
0,A,396.120589
1,B,348.804498


Снижение среднего чека: 11.94%


Добавляю средний чек в датафрейм `users_and_buyers`.

In [11]:
users_and_buyers = users_and_buyers.merge(avc, on='grp', how='inner')
display(users_and_buyers)

,grp,users_count,buyers,users_share,buyers_share,conversion,avg_cheque
0,A,76605,4279,0.499928,0.312199,0.055858,396.120589
1,B,76627,9427,0.500072,0.687801,0.123025,348.804498


Скидка была 30%, а средний чек снизился только на 11%. Единственное возможное объяснение этому — более высокая доля подписок на год (за 700 руб.) в экспериментальной группе, чем в контрольной группе.  Проверю это предположение.

In [12]:
price_summary_df = buyers_only_abtest.groupby(['grp'], as_index=True)['price'].value_counts().to_frame().rename(
    {'price': 'buyers'}, axis=1)
display(price_summary_df)

buyers
grp price         
A   200.0     3230
    1000.0    1049
B   140.0     5912
    700.0     3515

Для удобства группировки сделаю метки для подписок на месяц и на год и добавлю их в датафрейм.

In [13]:
price_summary_df = price_summary_df.reset_index()
# Этот маленький трюк подставляет в колонку subscription слова "month"
# или "year" в зависимости от цены. Если цена < 250, то речь идёт о подписке
# на месяц, иначе -- на год.
price_summary_df['subscription'] = price_summary_df.price.apply(lambda x: "month" if x<250  else "year")
# Меня интересует только количество покупателей, сгруппированное по 2 вложенным категориям.
price_summary_df = price_summary_df.set_index(['grp', 'subscription'])['buyers'].to_frame()
price_summary_df

buyers
grp subscription        
A   month           3230
    year            1049
B   month           5912
    year            3515

Теппрь рассчитаю доли месячной и годичной подписки в контрольной и экспериментальной группах.

In [14]:
a_mon = (price_summary_df.loc[('A', 'month')] / (price_summary_df.loc['A'].sum()))[0]
a_yr  = (price_summary_df.loc[('A', 'year')]  / (price_summary_df.loc['A'].sum()))[0]
b_mon = (price_summary_df.loc[('B', 'month')] / (price_summary_df.loc['B'].sum()))[0]
b_yr  = (price_summary_df.loc[('B', 'year')]  / (price_summary_df.loc['B'].sum()))[0]

print('''
В группе A {0:.1%} покупателей приобрели месячную подписку, а остальные {1:.1%} годичную.
В то же время в группе B {2:.1%} приобрели месячную подписку, а годичную -- {3:.1%}.
'''.format(a_mon, a_yr, b_mon, b_yr))


В группе A 75.5% покупателей приобрели месячную подписку, а остальные 24.5% годичную.
В то же время в группе B 62.7% приобрели месячную подписку, а годичную -- 37.3%.



Предположение подтверждается: в группе B существенно выше доля годичных подписок.

### Расчёт ARPU

В строгом смысле мы ARPU расчитать не можем, так как известна только прибыль, но не издержки.  Если принять издержки за ноль, то сложностей возникнуть не должно: в этом случае ARPU это средний чек, умноженный на конверсию (мы уже знаем, что на одного пользователя одна покупка). Всё нужное есть в датафрейме `users_and_buyers`:

!!! ссылочку надо.

In [15]:
display(users_and_buyers)

,grp,users_count,buyers,users_share,buyers_share,conversion,avg_cheque
0,A,76605,4279,0.499928,0.312199,0.055858,396.120589
1,B,76627,9427,0.500072,0.687801,0.123025,348.804498


In [16]:
users_and_buyers['arpu'] = users_and_buyers.conversion*users_and_buyers.avg_cheque
display(users_and_buyers)

,grp,users_count,buyers,users_share,buyers_share,conversion,avg_cheque,arpu
0,A,76605,4279,0.499928,0.312199,0.055858,396.120589,22.126493
1,B,76627,9427,0.500072,0.687801,0.123025,348.804498,42.911506


Считаем дополнительную выручку. Чисел мало, проще всего просто подставить их копипастом.

In [17]:
temp_df = users_and_buyers.set_index('grp')
b_group_old_revenue = temp_df.loc['B','users_count'] * temp_df.loc['A','arpu']
b_group_old_revenue
b_group_new_revenue = temp_df.loc['B','users_count'] * temp_df.loc['B','arpu']
b_group_new_revenue
b_group_additional_revenue = b_group_new_revenue - b_group_old_revenue
print('''При сохранении ARPU на уровне группы A группа B принесла бы {0:.0f} руб. прибыли.
Фактически она принесла {1:.0f} руб., то есть на {2:.0f} руб. больше.
'''.format(b_group_old_revenue, b_group_new_revenue, b_group_additional_revenue)
)


При сохранении ARPU на уровне группы A группа B принесла бы 1695487 руб. прибыли.
Фактически она принесла 3288180 руб., то есть на 1592693 руб. больше.



## Считаем статистическую значимость

Мы получили какие-то цифры (конверсию, ARPU, разницу доходов).  Но насколько уверенно можно утверждать, что это не результат случайности при выборке пользователей, а именно результат скидок?  Для этого нужно посчитать доверительный интервал и p-value, то есть вероятность случайного получения эффекта такой же или большей величины при выборке.
Доверительный интервал и p-value буду расчитывать только для конверсии, она оптимальна для такого расчёта, так как это пропорция. Вспоминаю [Guidelines for A/B Testing](https://hookedondata.org/guidelines-for-ab-testing/), и не поддаюсь соблазну считать статистику для других показателей (доли годовых подписок, например).
Для попыток расчёта параметров для покупателей есть ещё одно препятствие: покупателей существенно меньше, чем пользователей, поэтому велика вероятность получить высокие p-Value, то есть бесполезные цифры.

Расчитаю доверительные интервалы с 95% уверенности для конверсии в экспериментальной и контрольной группах.

Вспоминаем формулу (речь о пропорции, поэтому используем распределение Гаусса):

$$
p = \hat{p} \pm Z_{0.025} \times \sqrt{ {\hat{p}\,(1-\hat{p})\strut} \over {n} }
$$

Выражение с корнем — стандартная ошибка для выборочной пропорции.

Контрольная группа

$$
p_k = 4279/76605 \pm 1.96 \times \sqrt{ { 4279/76605 \,(1 - 4279/76605\strut) }\over 76605 }
\approx 0.056 \pm 8.297 \cdot 10^{-4}
\approx [ 0.054..0.057 ]
$$

Экспериментальная группа:

$$
p_e = 9427/76627 \pm 1.96 \times \sqrt { {9427/76627\,(1-9427/76627)\strut} \over 76627 } 
\approx 0.123 \pm 2.325\cdot10^{-3}
\approx [0.121..0.125]
$$

Доверительные интервалы не пересекаются (и далеки друг от друга, отличие больше чем в 2 раза), что даёт приличную уверенность в том, что конверсии в контрольной и экспериментальной группах действительно разные.  Но расчитаем и p-value.  Для этого необходимо зафиксировать нулевую и альтернативную гипотезы.
#### Гипотезы

* $H_0$: конверсии в экспериментальной и контрольной группах равны.
* $H_a$: конверсия в экспериментальной группе выше, чем в контрольной.

Переходим к Z-score для $p_e$.  Формула подразумевает стандартную ошибку разности
пропорций $p_k$ и $p_e$, но эту ошибку сначала нужно вычислить.  Мы знаем, что при 
вычитании двух _независимых_ случайных величин их вариации складываются, и есть основания
полагать, что наши выборки независимы: в них входят разные пользователи.  От вариации
легко перейти к стандартной ошибке (извлечением корня).

Вариация для $p_k$:

$$
Var_{p_k} =  \frac{ 4279/76605 \times (1 - 4279/76605\strut) }{76605} \approx 6.884 \cdot 10^{-7}
$$

Variance для пропорции $p_e$:
$$
Var_{p_e} = \frac{9427/76627 \times (1-9427/76627\strut)}{76627} \approx 1.408 \cdot 10^{-6}
$$

Отсюда стандартное отклонение разницы 

$$
se_{diff} = \sqrt{ Var_{p_e} + Var_{p_k}\strut } \approx 1.448 \cdot 10^{-3} 
$$

Z-score: 

$$
Z_\hat{p} = \frac{\hat{p} - p}{se_{diff}} \approx 46.27.
$$

Это очень высокий Z-score, ему соответствует крайне низкий p-value, практически нулевой (точности 32-битного целого на моём компьютере не хватает, чтобы его показать).

## Вывод: 

Подтверждена статистическая значимость изменения конверсии в группе B относительно группы A.

Попробую ещё библиотечную функцию (`ztest` из модуля `statsmodels.stats.weightstats`), чтобы посчитать Z-score и p-value:

### Расчёт Z-score и P-value функцией ztest из пакета statsmodels:

In [29]:
from statsmodels.stats.weightstats import ztest
# Небольшой хак: мы знаем, что int(False) = 0 и int(True) = 1
grp_A = abtest_df.loc[abtest_df.grp=='A', 'price'].notnull().astype(int)
grp_B = abtest_df.loc[abtest_df.grp=='B', 'price'].notnull().astype(int)
print(f"Группа A: {grp_A.sum()} покупателей из {len(grp_A)} пользователей.")
print(f"Группа B: {grp_B.sum()} покупателей из {len(grp_B)} пользователей.")
zscore, pvalue = ztest(grp_A, grp_B)
print(f"Z-score: {zscore:.3f}, p-value: {pvalue}")

Группа A: 4279 покупателей из 76605 пользователей.
Группа B: 9427 покупателей из 76627 пользователей.
Z-score: -46.386, P-Value: 0.0


Похоже, в ручном расчёте Z-score я несколько ошибся, но на конечный результат это не повлияло.